In [6]:
# Directories
import os

new_directory = r'c://Users//Fer//TESIS_ARCHIVOS//TESIS_AIRE//MP_Forecasting//aqi_forecasting//notebooks'
os.chdir(new_directory)

# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Training utils
import utils_xgboost

# Optuna
import optuna
import pickle

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO


# Metricas
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_absolute_percentage_error #MAPE
from sklearn.metrics import mean_squared_error #MSE, para RMSE: squared = False

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt
import seaborn as sn

# Advertencias
import warnings
warnings.filterwarnings("ignore")

In [12]:
df_train = pd.read_csv('datos/230127_train_ESTACIONES.csv', parse_dates=['FECHAHORA'])
df_test = pd.read_csv('datos/230127_test_ESTACIONES.csv', parse_dates=['FECHAHORA'])

# Datos a eliminar

 2   ANHO                    264960 non-null  int64         
 3   DIA                     264960 non-null  int64         
 4   MES                     264960 non-null  int64         
 5   HORA                    264960 non-null  int64         
 6   MINUTO                  264960 non-null  int64   
 
 13  DIA_TRAF_COD            264960 non-null  object        
 14  TRAFICO_COD             264960 non-null  object 
 
 16  TIPO_COD                264960 non-null  object 
 

 21  MEDICION_DIA            264960 non-null  int64         
 22  MP1_ANTERIOR            264960 non-null  float64       
 23  MP2_5_ANTERIOR          264960 non-null  float64       
 24  MP10_ANTERIOR           264960 non-null  float64       
 25  TEMPERATURA_PRONOSTICO  264960 non-null  float64       
 26  HUMEDAD_PRONOSTICO      264960 non-null  float64       
 27  PRESION_PRONOSTICO      264960 non-null  float64     

In [13]:
df_test.drop(['ANHO', 'DIA', 'MES', 'HORA', 'MINUTO', 'DIA_TRAF_COD', 'TRAFICO_COD', 'TIPO_COD',
              'MEDICION_DIA', 'MP1_ANTERIOR', 'MP2_5_ANTERIOR', 'MP10_ANTERIOR',
              'TEMPERATURA_PRONOSTICO', 'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO'], axis = 1, inplace = True)

df_train.drop(['ANHO', 'DIA', 'MES', 'HORA', 'MINUTO', 'DIA_TRAF_COD', 'TRAFICO_COD', 'TIPO_COD',
              'MEDICION_DIA', 'MP1_ANTERIOR', 'MP2_5_ANTERIOR', 'MP10_ANTERIOR',
              'TEMPERATURA_PRONOSTICO', 'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO'], axis = 1, inplace = True)

df_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1315882 entries, 0 to 1315881
Data columns (total 13 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   ESTACION     1315882 non-null  int64         
 1   FECHAHORA    1315882 non-null  datetime64[ns]
 2   MP1          1315882 non-null  float64       
 3   MP2_5        1315882 non-null  float64       
 4   MP10         1315882 non-null  float64       
 5   TEMPERATURA  1315882 non-null  float64       
 6   HUMEDAD      1315882 non-null  float64       
 7   PRESION      1315882 non-null  float64       
 8   TRAFICO      1315882 non-null  int64         
 9   TIPO         1315882 non-null  int64         
 10  DIA_SEM      1315882 non-null  int64         
 11  AQI_MP2_5    1315882 non-null  float64       
 12  AQI_MP10     1315882 non-null  float64       
dtypes: datetime64[ns](1), float64(8), int64(4)
memory usage: 130.5 MB


In [16]:
df_train_resampled = df_train.set_index('FECHAHORA', drop = True)
df_test_resampled = df_test.set_index('FECHAHORA', drop = True)

In [17]:
df_train_resampled.head()

,ESTACION,MP1,MP2_5,MP10,TEMPERATURA,HUMEDAD,PRESION,TRAFICO,TIPO,DIA_SEM,AQI_MP2_5,AQI_MP10
FECHAHORA,,,,,,,,,,,,
2019-12-30 13:00:00,1,2.839,3.989,4.599,38.0,43.0,994.8,1,1,0,72.0,25.0
2019-12-30 13:05:00,1,9.000,13.079,21.319,38.3,41.0,994.7,1,1,0,72.0,25.0
2019-12-30 13:10:00,1,1.969,2.369,13.170,38.0,40.8,994.6,1,1,0,72.0,25.0
2019-12-30 13:15:00,1,1.740,2.379,2.429,38.4,39.8,994.6,1,1,0,72.0,25.0
2019-12-30 13:20:00,1,2.710,4.119,7.710,38.7,39.5,994.7,1,1,0,72.0,25.0


In [21]:
lista = []
for i in range(1,11):
    df_aux = df_train_resampled[df_train_resampled['ESTACION'] == i]

    df_aux = df_aux.resample('60T').mean()
    lista.append(df_aux)

df_train_resampled = pd.concat(lista)

lista = []

for i in range(1,11):
    df_aux = df_test_resampled[df_test_resampled['ESTACION'] == i]

    df_aux = df_aux.resample('60T').mean()
    lista.append(df_aux)

df_test_resampled = pd.concat(lista)


df_test_resampled.head()




,ESTACION,MP1,MP2_5,MP10,TEMPERATURA,HUMEDAD,PRESION,TRAFICO,TIPO,DIA_SEM,AQI_MP2_5,AQI_MP10
FECHAHORA,,,,,,,,,,,,
2021-03-30 13:00:00,1.0,0.892273,1.095545,2.162000,28.190909,50.354545,1000.818182,1.0,1.0,1.0,53.0,14.0
2021-03-30 14:00:00,1.0,0.912500,1.162333,1.692750,28.100000,50.083333,1000.225000,1.0,1.0,1.0,53.0,14.0
2021-03-30 15:00:00,1.0,0.759500,0.945750,1.310917,27.808333,51.083333,999.841667,1.0,1.0,1.0,53.0,14.0
2021-03-30 16:00:00,1.0,1.027667,1.301083,2.559167,26.766667,54.433333,999.658333,2.0,2.0,1.0,53.0,13.5
2021-03-30 17:00:00,1.0,2.579917,3.206083,5.162500,24.808333,65.391667,999.641667,2.0,2.0,1.0,53.0,13.0


In [30]:
df_test_resampled['ANHO'] = df_test_resampled.index.year
df_test_resampled['MES'] = df_test_resampled.index.month
df_test_resampled['HORA'] = df_test_resampled.index.hour
df_test_resampled['MINUTO'] = df_test_resampled.index.minute
df_test_resampled['DIA'] = df_test_resampled.index.day


df_test_resampled.head()

,ESTACION,MP1,MP2_5,MP10,TEMPERATURA,HUMEDAD,PRESION,TRAFICO,TIPO,DIA_SEM,AQI_MP2_5,AQI_MP10,HORA,ANHO,MES,MINUTO,FECHAHORA,DIA
FECHAHORA,,,,,,,,,,,,,,,,,,
2021-03-30 13:00:00,1.0,0.892273,1.095545,2.162000,28.190909,50.354545,1000.818182,1.0,1.0,1.0,53.0,14.0,13,2021,3,0,2021-03-30 13:00:00,30
2021-03-30 14:00:00,1.0,0.912500,1.162333,1.692750,28.100000,50.083333,1000.225000,1.0,1.0,1.0,53.0,14.0,14,2021,3,0,2021-03-30 14:00:00,30
2021-03-30 15:00:00,1.0,0.759500,0.945750,1.310917,27.808333,51.083333,999.841667,1.0,1.0,1.0,53.0,14.0,15,2021,3,0,2021-03-30 15:00:00,30
2021-03-30 16:00:00,1.0,1.027667,1.301083,2.559167,26.766667,54.433333,999.658333,2.0,2.0,1.0,53.0,13.5,16,2021,3,0,2021-03-30 16:00:00,30
2021-03-30 17:00:00,1.0,2.579917,3.206083,5.162500,24.808333,65.391667,999.641667,2.0,2.0,1.0,53.0,13.0,17,2021,3,0,2021-03-30 17:00:00,30


In [31]:
df_train_resampled['ANHO'] = df_train_resampled.index.year
df_train_resampled['MES'] = df_train_resampled.index.month
df_train_resampled['HORA'] = df_train_resampled.index.hour
df_train_resampled['MINUTO'] = df_train_resampled.index.minute
df_train_resampled['DIA'] = df_train_resampled.index.day

df_train_resampled.head()

,ESTACION,MP1,MP2_5,MP10,TEMPERATURA,HUMEDAD,PRESION,TRAFICO,TIPO,DIA_SEM,AQI_MP2_5,AQI_MP10,ANHO,MES,HORA,MINUTO,DIA
FECHAHORA,,,,,,,,,,,,,,,,,
2019-12-30 13:00:00,1.0,3.257473,4.599460,7.976507,38.468000,39.516000,994.558000,1.0,1.0,0.0,72.000000,25.000000,2019,12,13,0,30
2019-12-30 14:00:00,1.0,2.221847,3.116000,5.936800,39.161333,34.489333,993.672000,1.0,1.0,0.0,72.000000,25.000000,2019,12,14,0,30
2019-12-30 15:00:00,1.0,2.159967,2.972800,5.542533,35.192667,41.394667,993.230667,1.0,1.0,0.0,72.000000,25.000000,2019,12,15,0,30
2019-12-30 16:00:00,1.0,3.620200,4.880387,8.760400,32.522667,45.682000,992.312667,2.0,2.0,0.0,72.933333,25.000000,2019,12,16,0,30
2019-12-30 17:00:00,1.0,5.094900,7.035600,12.224667,32.550000,44.436667,991.636667,2.0,2.0,0.0,74.000000,25.533333,2019,12,17,0,30


ValueError: cannot insert FECHAHORA, already exists

In [32]:
df_test_resampled.to_csv('datos/230325_test_resampled_1h.csv')
df_train_resampled.to_csv('datos/230325_train_resampled_1h.csv')